In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
# loading stopwords 
from nltk.corpus import stopwords
stopword = list(stopwords.words('english'))

In [3]:
len(stopword)

179

In [4]:
#load data 
TrainSet=pd.read_csv("../input/nlp-getting-started/train.csv")
TestSet=pd.read_csv("../input/nlp-getting-started/test.csv")
SubSet=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")

In [5]:
TrainSet.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# get text and label from TrainSet
Traintext=TrainSet['text'].values
Trainlabel=TrainSet['target'].values

In [7]:
Trainlabel.shape

(7613,)

In [8]:
Valt=Traintext[:1000]
Valla=Trainlabel[:1000]
Traint=Traintext[1000:]
Trainla=Trainlabel[1000:]

In [9]:
# remove stop words
def removeStopW(sent:list):
    sentence=[]
    for sen in sent:
        make=""
        for word in str(sen).lower().split():
            if word not in stopword:
                make=make+word+" "
        sentence.append(make.strip())
    return sentence

In [10]:
TraintR=removeStopW(list(Traint))
ValtR=removeStopW(list(Valt))


In [11]:
# working with deep learning model
import tensorflow as tf

In [12]:
# preprocessing data
oov_token='<OOV>'
Tokenizer=tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)

In [13]:
Tokenizer.fit_on_texts(TraintR)

In [14]:
len(Tokenizer.word_index)

20385

In [15]:
TrainSeq=Tokenizer.texts_to_sequences(TraintR)
ValSeq=Tokenizer.texts_to_sequences(ValtR)

In [16]:
# padding the sequence
Trainpad=tf.keras.preprocessing.sequence.pad_sequences(TrainSeq,padding='post')
Valpad=tf.keras.preprocessing.sequence.pad_sequences(ValSeq,padding='post',maxlen=Trainpad.shape[1])

In [17]:
Trainpad.shape[1]


28

In [18]:
# create a mode
model=tf.keras.Sequential(
[
    tf.keras.layers.Embedding(len(Tokenizer.word_index)+1,100,input_length=Trainpad.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100,dropout=0.2,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100,dropout=0.2,return_sequences=False)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(50,activation='relu'),
    tf.keras.layers.Dropout(0.01),
    tf.keras.layers.Dense(20, activation = "relu"),
    tf.keras.layers.Dense(1,activation='sigmoid')
    
])


2022-10-14 00:50:33.273728: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 28, 100)           2038600   
_________________________________________________________________
bidirectional (Bidirectional (None, 28, 200)           160800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               240800    
_________________________________________________________________
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                10050     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1

In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
model.fit(Trainpad,Trainla,validation_data=(Valpad,Valla),epochs=20,batch_size=64,callbacks=tf.keras.callbacks.EarlyStopping(patience=6))

2022-10-14 00:50:34.715455: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
104/104 [==============================] - 29s 198ms/step - loss: 0.5699 - accuracy: 0.6964 - val_loss: 0.4738 - val_accuracy: 0.7920
Epoch 2/20
104/104 [==============================] - 19s 179ms/step - loss: 0.2936 - accuracy: 0.8895 - val_loss: 0.6208 - val_accuracy: 0.7590
Epoch 3/20
104/104 [==============================] - 18s 169ms/step - loss: 0.1326 - accuracy: 0.9566 - val_loss: 0.7812 - val_accuracy: 0.6750
Epoch 4/20
104/104 [==============================] - 19s 182ms/step - loss: 0.0766 - accuracy: 0.9781 - val_loss: 0.7383 - val_accuracy: 0.7380
Epoch 5/20
104/104 [==============================] - 18s 172ms/step - loss: 0.0454 - accuracy: 0.9874 - val_loss: 0.9987 - val_accuracy: 0.7080
Epoch 6/20
104/104 [==============================] - 18s 178ms/step - loss: 0.0316 - accuracy: 0.9921 - val_loss: 0.9686 - val_accuracy: 0.7250
Epoch 7/20
104/104 [==============================] - 19s 183ms/step - loss: 0.0260 - accuracy: 0.9933 - val_loss: 0.9633 - val_ac

In [22]:
model.evaluate(Valpad,Valla)

32/32 [==============================] - 1s 35ms/step - loss: 0.9633 - accuracy: 0.7480


[0.9633233547210693, 0.7480000257492065]

In [23]:
SubSet

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [24]:
Testtext=TestSet['text'].values

In [25]:
Testtext=removeStopW(Testtext)

In [26]:
Testpad=tf.keras.preprocessing.sequence.pad_sequences(Tokenizer.texts_to_sequences(Testtext),padding='post',maxlen=Trainpad.shape[1])

In [27]:
predVal=model.predict(Testpad)

In [28]:
for i in range(predVal.shape[0]):
    SubSet['target'][i]=int(np.round(predVal[i]))
    


In [29]:
SubSet.to_csv("Submision.csv")